In [1]:
import os
import csv
from csv import reader
import glob
import random
import itertools

include_folders = ['gcc', 'linux', 'mysql']

dir_name = '/home/shared/PLDI21_model'
dataset_path = '../datasets/functionally-equiv-compilable/'
map_path = 'data/equal/path2tokens.tsv'

equal_tsv = 'data/equal/path_train_eq.tsv'
ne_1_tsv = 'data/equal/path_train_ne_1.tsv'
ne_2_tsv = 'data/equal/path_train_ne_2.tsv'

id_equal_tsv = 'data/equal/token_train_eq.tsv'
id_ne_1_tsv = 'data/equal/token_train_ne_1.tsv'
id_ne_2_tsv = 'data/equal/token_train_ne_2.tsv'

combined_eq_ne_1_tsv = 'data/equal/combined_eq_ne_1.tsv'
combined_eq_ne_2_tsv = 'data/equal/combined_eq_ne_2.tsv'

ratio_eq_ne_1_tsv = 'data/equal/ratio_eq_ne_1.tsv'
ratio_eq_ne_2_tsv = 'data/equal/ratio_eq_ne_2.tsv'

## Replace Path by Token Seq

In [2]:
path_map = {}
with open(map_path, 'r') as f_input:   
    tsv_reader = reader(f_input, delimiter='\t')
    header = next(tsv_reader)
    if header != None:
      for row in tsv_reader:
        path_map[row[0]] = row[1]
print('tsv size:', len(path_map))

tsv size: 2596


In [3]:
count = 0
dirs = [os.path.join(dataset_path, o) for o in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path,o)) \
        and o in include_folders]
for dir in dirs:
      sub_dirs =  [os.path.join(dir, o) for o in os.listdir(dir) if o != 'include']
      for sub_dir in sub_dirs:
          sub_subdirs = [os.path.join(sub_dir, o) for o in os.listdir(sub_dir)]
          for sub_subdir in sub_subdirs:
              files = glob.glob(sub_subdir + '/*.c')
              count += len(files)
print('file count:', count)

file count: 2596


## R/W EQ examples into TSV

In [4]:
train_equal_size = 0
with open(equal_tsv, 'w', newline='') as f_output, open(id_equal_tsv, 'w', newline='') as id_output:
    tsv_output = csv.writer(f_output, delimiter='\t')
    tsv_output.writerow(['file_1', 'file_2', 'label'])
    
    id_output = csv.writer(id_output, delimiter='\t')
    id_output.writerow(['id_1', 'id_2', 'label'])

    dirs = [os.path.join(dataset_path, o) for o in os.listdir(dataset_path) 
                        if os.path.isdir(os.path.join(dataset_path,o)) and o in include_folders]
    for dir in dirs:
        eq_dirs =  [os.path.join(dir, o) for o in os.listdir(dir) if '_eq'in o]
        for eq_dir in eq_dirs:
            eq_subdirs = [os.path.join(eq_dir, o) for o in os.listdir(eq_dir)]
            for eq_subdir in eq_subdirs:
                files = glob.glob(eq_subdir + '/*.c')
                pairs = list(itertools.combinations(files, 2))
                for pair in pairs:
                    file_1 = pair[0].replace('..', dir_name).replace('\\', '/')
                    file_2 = pair[1].replace('..', dir_name).replace('\\', '/')
                    id_1 = path_map[file_1]
                    id_2 = path_map[file_2]
                    tsv_output.writerow([file_1, file_2, '1'])
                    id_output.writerow([id_1, id_2, '1'])
                    train_equal_size += 1
print('train_equal_size:', train_equal_size)

train_equal_size: 1114


## R/W NE_1 examples into TSV

In [5]:
train_ne1_size = 0
with open(ne_1_tsv, 'w', newline='') as f_output, open(id_ne_1_tsv, 'w', newline='') as id_output:
    tsv_output = csv.writer(f_output, delimiter='\t')
    tsv_output.writerow(['file_1', 'file_2', 'label'])
    
    id_output = csv.writer(id_output, delimiter='\t')
    id_output.writerow(['id_1', 'id_2', 'label'])

    dirs = [os.path.join(dataset_path, o) for o in os.listdir(dataset_path) 
                            if os.path.isdir(os.path.join(dataset_path,o)) and o in include_folders]
    for dir in dirs:
        eq_dirs = [os.path.join(dir, o) for o in os.listdir(dir) if '_eq'in o]
        ne_dirs = [os.path.join(dir, o) for o in os.listdir(dir) if '_ne_1'in o]
        for ne_dir in ne_dirs:
            ne_subdirs = [os.path.join(ne_dir, o) for o in os.listdir(ne_dir)]
            for ne_subdir in ne_subdirs:
                ne_files = glob.glob(ne_subdir + '/*.c')
                ne_subdir_name = ne_subdir.split('\\')[-1]
                for eq_dir in eq_dirs:
                  eq_subdir = [os.path.join(eq_dir, o) for o in os.listdir(eq_dir) if ne_subdir_name in o][0]
                eq_files = glob.glob(eq_subdir + '/*.c')
                for ne_file in ne_files:
                  for eq_file in eq_files:
                    file_1 = ne_file.replace('..', dir_name).replace('\\', '/')
                    file_2 = eq_file.replace('..', dir_name).replace('\\', '/')
                    id_1 = path_map[file_1]
                    id_2 = path_map[file_2]
                    tsv_output.writerow([file_1, file_2, '0'])
                    id_output.writerow([id_1, id_2, '0'])
                    train_ne1_size += 1
print('train_ne1_size:', train_ne1_size)

train_ne1_size: 2553


## R/W NE_2 examples into TSV

In [6]:
train_ne2_size = 0
with open(ne_2_tsv, 'w', newline='') as f_output, open(id_ne_2_tsv, 'w', newline='') as id_output:
    tsv_output = csv.writer(f_output, delimiter='\t')
    tsv_output.writerow(['file_1', 'file_2', 'label'])
    
    id_output = csv.writer(id_output, delimiter='\t')
    id_output.writerow(['id_1', 'id_2', 'label'])
    
    dirs = [os.path.join(dataset_path, o) for o in os.listdir(dataset_path) 
                            if os.path.isdir(os.path.join(dataset_path,o)) and o in include_folders]
    for dir in dirs:
        eq_dirs = [os.path.join(dir, o) for o in os.listdir(dir) if '_eq'in o]
        ne_dirs = [os.path.join(dir, o) for o in os.listdir(dir) if '_ne_2'in o]
        for ne_dir in ne_dirs:
            ne_subdirs = [os.path.join(ne_dir, o) for o in os.listdir(ne_dir)]
            for ne_subdir in ne_subdirs:
                ne_files = glob.glob(ne_subdir + '/*.c')
                ne_subdir_name = ne_subdir.split('\\')[-1]
                for eq_dir in eq_dirs:
                  eq_subdir = [os.path.join(eq_dir, o) for o in os.listdir(eq_dir) if ne_subdir_name in o][0]
                eq_files = glob.glob(eq_subdir + '/*.c')
                for ne_file in ne_files:
                  for eq_file in eq_files:
                    file_1 = ne_file.replace('..', dir_name).replace('\\', '/')
                    file_2 = eq_file.replace('..', dir_name).replace('\\', '/')
                    id_1 = path_map[file_1]
                    id_2 = path_map[file_2]
                    tsv_output.writerow([file_1, file_2, '0'])
                    id_output.writerow([id_1, id_2, '0'])
                    train_ne2_size += 1
print('train_ne2_size:', train_ne2_size)

train_ne2_size: 1227


## All combined datasets 

In [7]:
with open(combined_eq_ne_1_tsv, 'w', newline='') as f_ne_1, open(combined_eq_ne_2_tsv, 'w', newline='') as f_ne_2:
  tsv_output_1 = csv.writer(f_ne_1, delimiter='\t')
  tsv_output_2 = csv.writer(f_ne_2, delimiter='\t')
  
  tsv_output_1.writerow(['code_1', 'code_2', 'label'])
  tsv_output_2.writerow(['code_1', 'code_2', 'label'])
 
  with open(id_equal_tsv, 'r') as f_input:   
    tsv_reader = reader(f_input, delimiter='\t')
    header = next(tsv_reader)
    if header != None:
      for row in tsv_reader:
        tsv_output_1.writerow(row)
        tsv_output_2.writerow(row)
        
  with open(id_ne_1_tsv, 'r') as f_input:   
    tsv_reader = reader(f_input, delimiter='\t')
    header = next(tsv_reader)
    if header != None:
      for row in tsv_reader:
        tsv_output_1.writerow(row)
        
  with open(id_ne_2_tsv, 'r') as f_input:   
    tsv_reader = reader(f_input, delimiter='\t')
    header = next(tsv_reader)
    if header != None:
      for row in tsv_reader:
        tsv_output_2.writerow(row)

## 1:1 ratio datasets 

In [8]:
with open(ratio_eq_ne_1_tsv, 'w', newline='') as f_ne_1, open(ratio_eq_ne_2_tsv, 'w', newline='') as f_ne_2:
  tsv_output_1 = csv.writer(f_ne_1, delimiter='\t')
  tsv_output_2 = csv.writer(f_ne_2, delimiter='\t')
  
  tsv_output_1.writerow(['code_1', 'code_2', 'label'])
  tsv_output_2.writerow(['code_1', 'code_2', 'label'])
 
  with open(id_equal_tsv, 'r') as f_input:   
    tsv_reader = reader(f_input, delimiter='\t')
    header = next(tsv_reader)
    if header != None:
      count = 0
      for row in tsv_reader:
        if count < train_equal_size:
          tsv_output_1.writerow(row)
          tsv_output_2.writerow(row)
          count += 1
        
  with open(id_ne_1_tsv, 'r') as f_input:   
    tsv_reader = reader(f_input, delimiter='\t')
    header = next(tsv_reader)
    if header != None:
      count = 0
      for row in tsv_reader:
        if count < train_equal_size:
          tsv_output_1.writerow(row)
          count += 1
        
  with open(id_ne_2_tsv, 'r') as f_input:   
    tsv_reader = reader(f_input, delimiter='\t')
    header = next(tsv_reader)
    if header != None:
      count = 0
      for row in tsv_reader:
        if count < train_equal_size:
          tsv_output_2.writerow(row)
          count += 1

## Shuffle & Divide Dataset into Train/Dev/Test: 7/1.5/1.5

In [11]:
def shuffle_and_divide(fpath):
  if (fpath == combined_eq_ne_1_tsv):
    outpath = 'data/equal/equal_dataset_combined_1/'
  elif (fpath == combined_eq_ne_2_tsv):
    outpath = 'data/equal/equal_dataset_combined_2/'
  elif (fpath == ratio_eq_ne_1_tsv):
    outpath = 'data/equal/equal_dataset_ratio_1/'
  else:
    outpath = 'data/equal/equal_dataset_ratio_2/'

  if not os.path.exists(os.path.dirname(outpath)):
    try:
      os.makedirs(os.path.dirname(outpath))
    except OSError as exc:
      if exc.errno != errno.EEXIST:
        raise

  with open(outpath+'train.tsv', 'w', newline='') as f_train, open(outpath+'dev.tsv', 'w', newline='') as f_dev, open(outpath+'test.tsv','w', newline='') as f_test:
      tsv_train = csv.writer(f_train, delimiter='\t')
      tsv_dev = csv.writer(f_dev, delimiter='\t')
      tsv_test = csv.writer(f_test, delimiter='\t')

      tsv_train.writerow(['code_1', 'code_2', 'label'])
      tsv_dev.writerow(['code_1', 'code_2', 'label'])
      tsv_test.writerow(['code_1', 'code_2', 'label'])

      fpath_shuffled = os.path.splitext(fpath)[0] + '_shuffled.tsv'
      with open(fpath, 'r') as f_read:
        data = f_read.readlines()
        header, rows = data[0], data[1:]
        random.shuffle(rows)
      with open(fpath_shuffled ,'w') as f_write:
        f_write.write(''.join([header]+rows))
      
      with open(fpath_shuffled, 'r') as f_combined, open(outpath+'test_labels.txt' ,'w') as f_labels:
        tsv_reader = reader(f_combined, delimiter='\t')
        header = next(tsv_reader)
        if header != None:
          i = 0
          for row in tsv_reader:
            if i <= 0.7 * len(rows):
              tsv_train.writerow(row)
            elif i <= 0.85 * len(rows):
              tsv_dev.writerow(row)
            else:
              tsv_test.writerow(row)
              f_labels.write(row[-1] + ' ')
            i += 1

In [12]:
for file in [combined_eq_ne_1_tsv, combined_eq_ne_2_tsv, ratio_eq_ne_1_tsv, ratio_eq_ne_2_tsv]:
  shuffle_and_divide(file)